In [1]:
import pandas as pd
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

In [2]:
data = pd.read_csv("daily_eur_usd.csv").iloc[::-1].reset_index(drop=True)

In [3]:
data

,date,open,high,low,close
0,2002-01-28,0.8630,0.8663,0.8567,0.8610
1,2002-01-29,0.8611,0.8656,0.8587,0.8644
2,2002-01-30,0.8642,0.8674,0.8597,0.8614
3,2002-01-31,0.8613,0.8646,0.8575,0.8579
4,2002-02-01,0.8586,0.8642,0.8560,0.8614
...,...,...,...,...,...
4995,2021-03-23,1.1934,1.1941,1.1840,1.1849
4996,2021-03-24,1.1849,1.1852,1.1808,1.1812
4997,2021-03-25,1.1812,1.1828,1.1760,1.1768
4998,2021-03-26,1.1768,1.1805,1.1766,1.1793


In [4]:
dn = data[1:].rename(columns={'open':'next_open', 'high':'next_high', 'low':'next_low', 'close':'next_close'}).reset_index(drop=True)
dn = dn.drop(['date'], axis=1)
dn

,next_open,next_high,next_low,next_close
0,0.8611,0.8656,0.8587,0.8644
1,0.8642,0.8674,0.8597,0.8614
2,0.8613,0.8646,0.8575,0.8579
3,0.8586,0.8642,0.8560,0.8614
4,0.8620,0.8711,0.8603,0.8700
...,...,...,...,...
4994,1.1934,1.1941,1.1840,1.1849
4995,1.1849,1.1852,1.1808,1.1812
4996,1.1812,1.1828,1.1760,1.1768
4997,1.1768,1.1805,1.1766,1.1793


In [5]:
res = pd.concat([data, dn],axis=1)[:-1]

In [6]:
res

,date,open,high,low,close,next_open,next_high,next_low,next_close
0,2002-01-28,0.8630,0.8663,0.8567,0.8610,0.8611,0.8656,0.8587,0.8644
1,2002-01-29,0.8611,0.8656,0.8587,0.8644,0.8642,0.8674,0.8597,0.8614
2,2002-01-30,0.8642,0.8674,0.8597,0.8614,0.8613,0.8646,0.8575,0.8579
3,2002-01-31,0.8613,0.8646,0.8575,0.8579,0.8586,0.8642,0.8560,0.8614
4,2002-02-01,0.8586,0.8642,0.8560,0.8614,0.8620,0.8711,0.8603,0.8700
...,...,...,...,...,...,...,...,...,...
4994,2021-03-22,1.1880,1.1947,1.1873,1.1932,1.1934,1.1941,1.1840,1.1849
4995,2021-03-23,1.1934,1.1941,1.1840,1.1849,1.1849,1.1852,1.1808,1.1812
4996,2021-03-24,1.1849,1.1852,1.1808,1.1812,1.1812,1.1828,1.1760,1.1768
4997,2021-03-25,1.1812,1.1828,1.1760,1.1768,1.1768,1.1805,1.1766,1.1793


In [7]:
# Drop last 9 data points so we have a multiple of 10
X, Y = res[:-9].open, res[:-9].next_open

In [8]:
model = Sequential()
model.add(LSTM(80, activation='relu', input_shape=(10,1)))
model.add(Dense(10, activation='relu'))
model.add(Dense(10))
model.compile(optimizer='adam', loss='mse')
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 80)                26240     
_________________________________________________________________
dense (Dense)                (None, 10)                810       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
Total params: 27,160
Trainable params: 27,160
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
X = X.values.reshape(len(X) // 10, 10, 1)
Y = Y.values.reshape(len(Y) // 10, 10, 1)

In [10]:
model.fit(X, Y, epochs=50, validation_split=0.2, batch_size=5)

Epoch 1/50
80/80 [==============================] - 4s 28ms/step - loss: 1.2546 - val_loss: 0.3235
Epoch 2/50
80/80 [==============================] - 1s 18ms/step - loss: 0.2977 - val_loss: 0.0926
Epoch 3/50
80/80 [==============================] - 1s 18ms/step - loss: 0.0684 - val_loss: 0.0089
Epoch 4/50
80/80 [==============================] - 2s 19ms/step - loss: 0.0076 - val_loss: 0.0013
Epoch 5/50
80/80 [==============================] - 2s 20ms/step - loss: 8.5871e-04 - val_loss: 1.1336e-04
Epoch 6/50
80/80 [==============================] - 1s 18ms/step - loss: 3.4797e-04 - val_loss: 2.6035e-04
Epoch 7/50
80/80 [==============================] - 2s 19ms/step - loss: 3.2934e-04 - val_loss: 1.5130e-04
Epoch 8/50
80/80 [==============================] - 2s 19ms/step - loss: 2.9089e-04 - val_loss: 1.4806e-04
Epoch 9/50
80/80 [==============================] - 1s 18ms/step - loss: 2.9705e-04 - val_loss: 1.1001e-04
Epoch 10/50
80/80 [==============================] - 1s 19ms/step - l

In [11]:
model.predict(X[-1:])

array([[1.2034519, 1.1971551, 1.2030754, 1.2022582, 1.1925395, 1.1993794,
        1.2039878, 1.1986665, 1.1971697, 1.1913819]], dtype=float32)

In [12]:
Y[-1:]

array([[[1.2089],
        [1.2062],
        [1.1968],
        [1.1917],
        [1.1846],
        [1.1899],
        [1.1929],
        [1.1985],
        [1.1949],
        [1.1928]]])